<a href="https://www.kaggle.com/code/hli111111/chao-detection-lyapunov-spetral-robustness-test?scriptVersionId=148701910" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


# Goal

Detect Chaos from series/ model

See which var and transformation permits a non-chaotic map from $X[t_{i},...t_{i+n}]$ to $X[t_{i+k},...,t_{i+k+n}]$

We do it for series data, but can also do any deterministic map. Although the inductive is not alway proper for this purpose.


# Lyapunov factor & spectral


Wolf's algorithm, which is a method for calculating the largest Lyapunov exponent from a time series. The largest Lyapunov exponent is a measure of the average exponential rate of divergence (or convergence) of close trajectories in a system, making it a useful metric for understanding chaos in dynamical systems.



This method, and the notion of Lyapunov, is referenced at

https://home.cs.colorado.edu/~lizb/chaos/wolf-notes.pdf

What is it?

We want to know whether the metric X at $t_1$ is  predictable from $t_0$ if deterministic at some space (T,d)

One simple case is to think that there is a *deterministic* map 

$$X_{t_1} = F(X_{t_0})$$
you want a model M to approximate F.
For sure, not all systems are like that, we can do

$$X_{t_1} = F(X_{t_0}) + \epsilon $$

and suggest epsilon is small compared to $\frac {\partial F} {\partial x}$

**of course this is not good enough either**
a feature engineering can do


$$X_{t_1} = T^{-1}(F(T(X_{t_0} + \epsilon))) $$

where T can be some feature transformations, e.g. adding, deleting columns.

**of course, you can add multi shot**, which means you can add an algorithm 
for example

$$A_k \rightarrow M_j \rightarrow F$$ 

**in this way, you can reselect model when it is defective. but if the prediction changes A, you lose**


So, this notebook is not a panacea.


Want to see if F is time invariant and non-chaotic.
Want to see if an A is time invariant(of course this will be hard),

**this only shows whether chaotic, and how, you need other note to see whether you can select an A that generalizes**

## Sensitivity

How much result changes w.r.t. error in par. In this case $\frac {\partial X(t)} {\partial X(t_0)}$, or $\frac {\partial \text{error in result}} {\partial \text{error in start time}}$

If the D.S. is deterministic, can use **chain rule** to multiply a number of partial derivatives together, this idea forms **Lyapunv spectral**.
In this case we want the larges L-factor in the spectral.

## Bifurcation

How solution topology changes w.r.t. parameters, usually denoted p or $\theta$  in some stats books.


## Conclusion

wait for finish.


In [2]:
import numpy as np

def wolf_algorithm(time_series, tau, m, r, T, verbose = 0):
    """
    Calculates the largest Lyapunov exponent using Wolf's algorithm.

    Parameters:
    - time_series: Input time series data
    - tau: Time delay
    - m: Embedding dimension
    - r: Threshold for close points
    - T: Max time over which to follow a pair of trajectories

    Returns:
    Largest Lyapunov exponent estimate.
    """

    N = len(time_series)
    M = N - (m - 1) * tau

    # Phase space reconstruction using delay embedding
    def embed(series, m, tau):
        return np.array([series[i:i+m*tau:tau] for i in range(0, N - (m - 1) * tau)])

    Y = embed(time_series, m, tau)

    def distance(y1, y2):
        return np.linalg.norm(np.nan_to_num(np.array(y1) - np.array(y2)))

    sums = 0
    count = 0

    for i in range(M):
        
        if verbose >1:
            if i % 10 ==0:
                print("On subsequence", i)
                print("Count total", count)
        yi = Y[i]

        # Find closest trajectory point j
        min_dist = np.inf
        j_star = -1
        for j in range(M):
            if abs(j - i) > T:  # Avoid temporally close points
                d = distance(yi, Y[j])
                if d < min_dist:
                    min_dist = d
                    j_star = j

        # If the closest point is closer than threshold r
        if min_dist < r:
            # Sum over the divergence of the trajectories
            for t in range(T):
                if i + t < M and j_star + t < M:
                    d_t = distance(Y[i + t], Y[j_star + t])
                    if d_t != 0:
                        sums += np.log(d_t / min_dist)
                        count += 1

    # Return the largest Lyapunov exponent
    if count == 0:
        return 0
        "no close enough trajectories are found"
    else:
        return sums / (T * count)

# Test the algorithm
time_series_data = np.sin(np.linspace(0, 50 * np.pi, 5000))  # Example sinusoidal data for testing
lyapunov_exp = wolf_algorithm(time_series_data, tau=10, m=2, r=0.1, T=10)
print(f"Largest Lyapunov Exponent: {lyapunov_exp}")


Largest Lyapunov Exponent: 1.037975057137938e-05


In [3]:
import pandas as pd
df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")

In [4]:
df = df.drop('row_id', axis = 1)

In [5]:
df.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id'],
      dtype='object')

In [6]:
df_wide = df.pivot_table(index = "date_id", columns = "stock_id",).reset_index()

In [7]:
df_wide.columns

MultiIndex([(  'date_id',  ''),
            ('ask_price',   0),
            ('ask_price',   1),
            ('ask_price',   2),
            ('ask_price',   3),
            ('ask_price',   4),
            ('ask_price',   5),
            ('ask_price',   6),
            ('ask_price',   7),
            ('ask_price',   8),
            ...
            (      'wap', 190),
            (      'wap', 191),
            (      'wap', 192),
            (      'wap', 193),
            (      'wap', 194),
            (      'wap', 195),
            (      'wap', 196),
            (      'wap', 197),
            (      'wap', 198),
            (      'wap', 199)],
           names=[None, 'stock_id'], length=2801)

In [8]:
df_wide_usable = df_wide.dropna(axis = 1)

# Convert column names to strings
df_wide_usable.columns = [str(col) for col in df_wide_usable.columns]

In [9]:

import pandas as pd

def get_categorical_and_numerical_columns(df):
    """
    Get the names of categorical and numerical columns from a DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame.

    Returns:
    categorical_columns (list): List of categorical column names.
    numerical_columns (list): List of numerical column names.
    """
    categorical_columns = df.select_dtypes(exclude=['number']).columns.tolist()
    numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
    
    return categorical_columns, numerical_columns

# Example usage:
# Assuming you have a DataFrame named 'my_df'
categorical_cols, numerical_cols = get_categorical_and_numerical_columns(df_wide_usable)
print("Categorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)




import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Create transformers for categorical and numerical columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create transformers for categorical and numerical columns
categorical_transformer_impute = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
])


numerical_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(max_iter=10, random_state=0)),
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, numerical_cols)
    ])

Categorical columns: []
Numerical columns: ["('date_id', '')", "('ask_price', 0)", "('ask_price', 1)", "('ask_price', 2)", "('ask_price', 3)", "('ask_price', 4)", "('ask_price', 5)", "('ask_price', 6)", "('ask_price', 7)", "('ask_price', 8)", "('ask_price', 9)", "('ask_price', 10)", "('ask_price', 11)", "('ask_price', 12)", "('ask_price', 13)", "('ask_price', 14)", "('ask_price', 15)", "('ask_price', 16)", "('ask_price', 17)", "('ask_price', 18)", "('ask_price', 20)", "('ask_price', 21)", "('ask_price', 22)", "('ask_price', 23)", "('ask_price', 24)", "('ask_price', 25)", "('ask_price', 26)", "('ask_price', 27)", "('ask_price', 28)", "('ask_price', 29)", "('ask_price', 30)", "('ask_price', 31)", "('ask_price', 32)", "('ask_price', 33)", "('ask_price', 34)", "('ask_price', 35)", "('ask_price', 36)", "('ask_price', 37)", "('ask_price', 38)", "('ask_price', 39)", "('ask_price', 40)", "('ask_price', 41)", "('ask_price', 42)", "('ask_price', 43)", "('ask_price', 44)", "('ask_price', 45)", "(

In [10]:
del df, df_wide
import gc
gc.collect()

0

In [11]:
df_wide_usable.columns[0]

"('date_id', '')"


# Create the final preprocessing pipeline
preprocessing_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])


time_series_df = preprocessing_pipeline.fit_transform(df_wide_usable)

In [12]:
df_mean = df_wide_usable.mean()
df_std = df_wide_usable.std()

normalized_df=(df_wide_usable-df_mean)/df_std

In [13]:
len(normalized_df.columns)

2611

Here to judge whether 2 snippnets of time series are close enough, we use normalized ones.

In [14]:
# Test the algorithm
time_series_data = normalized_df.select_dtypes(include='number').values.tolist()
lyapunov_exp = wolf_algorithm(time_series_data, tau=2, m=2, r=100, T=10, verbose = 2)
print(f"Largest Lyapunov Exponent: {lyapunov_exp}")


On subsequence 0
Count total 0
On subsequence 10
Count total 100
On subsequence 20
Count total 200
On subsequence 30
Count total 300
On subsequence 40
Count total 380
On subsequence 50
Count total 480
On subsequence 60
Count total 580
On subsequence 70
Count total 680
On subsequence 80
Count total 780
On subsequence 90
Count total 860
On subsequence 100
Count total 940
On subsequence 110
Count total 1040
On subsequence 120
Count total 1140
On subsequence 130
Count total 1240
On subsequence 140
Count total 1340
On subsequence 150
Count total 1440
On subsequence 160
Count total 1520
On subsequence 170
Count total 1620
On subsequence 180
Count total 1720
On subsequence 190
Count total 1820
On subsequence 200
Count total 1920
On subsequence 210
Count total 2000
On subsequence 220
Count total 2090
On subsequence 230
Count total 2160
On subsequence 240
Count total 2260
On subsequence 250
Count total 2360
On subsequence 260
Count total 2460
On subsequence 270
Count total 2560
On subsequence 2

In [15]:
lyapunov_exps_ests = [wolf_algorithm(time_series_data, tau=2, m=2, r=r_i, T=10, verbose = 0) for r_i in np.linspace(10,200,10)]


In [16]:
lyapunov_exps_ests

[0,
 0,
 0,
 0.018347500716528103,
 0.015334615775054494,
 0.014472626216702112,
 0.013201001126557026,
 0.013114521865343794,
 0.013114521865343794,
 0.013114521865343794]

In [17]:
np.linalg.norm(np.nan_to_num(np.array(time_series_data[0]) - np.array(time_series_data[1])))

50.448166829419094

## A interpolated F with a sample distribution?

If F is in T,d and is time invariant, we do not have a x_0 but we have a assumed a stable F, here we use a F with good cross validation score

**by no means I suggest this is the one with least sample/ inductive bias**



In [18]:
import numpy as np
from math import log

def generate_trajectory(f, x0, T):
    """Generate a trajectory using the deterministic map f."""
    trajectory = [x0]
    x = x0
    for _ in range(T - 1):
        x = f(x)
        trajectory.append(x)
    return trajectory

def wolf_algorithm_for_map(f, sample_x_from_S, d, n, T, r):
    """
    Compute the Lyapunov exponent for a deterministic map.
    
    f: Deterministic map.
    sample_x_from_S: Function to sample points from S.
    d: Distance function.
    n: Number of trajectories.
    T: Length of each trajectory.
    r: Distance threshold.
    """
    # Generate n trajectories of length T
    trajectories = [generate_trajectory(f, sample_x_from_S(), T) for _ in range(n)]

    sums = 0.0
    count = 0

    for i in range(n):
        xi = trajectories[i]
        min_dist = float('inf')
        j_star = -1

        for j in range(n):
            if i != j:  # Avoid comparing the trajectory with itself
                dist_ij = d(xi[0], trajectories[j][0])  # Distance at the start of the trajectory
                if dist_ij < min_dist:
                    min_dist = dist_ij
                    j_star = j

        if min_dist < r:
            for t in range(T):
                dist_t = d(xi[t], trajectories[j_star][t])
                if dist_t != 0:
                    sums += log(dist_t / min_dist)
                    count += 1

    if count == 0:
        return "No close enough point was found."
    else:
        return sums / count

# Example usage
def f(x):
    return 4*x*(1-x)  # Logistic map

def sample_x_from_S():
    return np.random.rand()  # Uniformly sample from [0, 1]

def distance(x1, x2):
    return abs(x1 - x2)

lyapunov_exp = wolf_algorithm_for_map(f, sample_x_from_S, distance, n=1000, T=50, r=0.01)
print(f"Largest Lyapunov Exponent: {lyapunov_exp}")


Largest Lyapunov Exponent: 5.999180922553409
